## Load packages

In [ ]:
# Classifier imports
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split #for creating a hold-out sample

import sklearn #for building models
import sklearn.ensemble #for building models
import pylab as plt
import pandas as pd
import numpy as np

from joblib import dump, load

## Read in data

In [ ]:
# Import paths
original_data = pd.read_csv("datasets/LISS_example_input_data.csv", encoding="cp1252") ##encoding="ISO-8859-1"

In [ ]:
outcome = pd.read_csv("datasets/LISS_example_groundtruth_data.csv")
# Drop observations where the outcome is missing
y_isna = outcome['new_child'].isnull()
data = original_data.loc[~y_isna]
outcome = outcome.loc[~y_isna]

## Select variables

In [ ]:
#check nas for each group
keepcols = ['positie2019','positie2018','gebjaar', 'geslacht','aantalhh2019', 'sted2019', 
            'nettohh_f2019', 'oplmet2019', 'herkomstgroep2019', 'cf19l128','cf19l129',
            'cf19l130', 'cf19l131','cf19l132','woning2019', 'woning2018', 
            'cf19l456', 'cf19l457', 'cf19l458', 'cf19l459', 'cw19l522', 'cr19l143', 
            'cf19l483', 'cf19l484', 'cf19l485', 'cf19l486', 'cf19l487', 'cf19l488',
           'wave2008', 'wave2014', 'wave2019','aantalki2017','aantalki2018',
            'partner2018','partner2019', 'belbezig2019','belbezig2018',
           'ch19l004','ch19l014','ch19l178','ch19l219',
            'cp19k118', 'cp19k021', 'cp19k056']#
#keepcols=['herkomstgroep2010', 'herkomstgroep2011',
#'herkomstgroep2012', 'herkomstgroep2013', 'herkomstgroep2014',
#'herkomstgroep2015', 'herkomstgroep2016', 'herkomstgroep2017',
#'herkomstgroep2018', 'herkomstgroep2019']

#keepcols = ['aantalki2007','aantalki2008','aantalki2009','aantalki2010','aantalki2011','aantalki2012',
            #'aantalki2013', 'aantalki2014', 'aantalki2015', 'aantalki2016', 'aantalki2017', 
            #'aantalki2018', 'aantalki2019']
data2 = data.loc[:, keepcols]
data2.isna().sum()
#data2['ch19l178'].value_counts()

In [ ]:
#Select features from kbset
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.datasets import load_iris
#from sklearn.feature_selection import SelectFromModel
#clf = ExtraTreesClassifier(n_estimators=50)
#clf = clf.fit(X_train, y_train)
#clf.feature_importances_  

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2

#from sklearn.preprocessing import OneHotEncoder
#encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")  
#pandas is just to set output to dataframe
#categorical_columns_selector = selector(dtype_include=object) #create selector for columns with type "object"
#categorical_columns = categorical_columns_selector(data)
#encoder.fit_transform(data[categorical_columns]) 

#X, y = load_digits(return_X_y=True)
#X_new = SelectKBest(chi2, k=20).fit_transform(data, outcome['new_child'])
#X_new.shape
dict_kids = {'None': 0, 'One child': 1, 'Two children': 2, 'Three children': 3, 'Four children': 4, 'Five children': 5, 'Six children': 6}
data["aantalki2018"] = data["aantalki2018"].map(dict_kids)

feature_test = Pipeline([
               ("preprocess", preprocessor),
               ("classifier", SelectKBest(chi2, k=50)) # LogisticRegression(max_iter=500)
               ]) 
feature_test.fit(data,outcome['new_child'])

#2nd way
data.columns[feature_test['classifier'].get_support(indices=True)].tolist()
#feature_test["classifier"].get_feature_names_out()
#feature_test["classifier"].columns[selector.get_support(indices=True)]

In [ ]:
#Select features from ExtraTreesClassifier

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
#clf = ExtraTreesClassifier(n_estimators=50)
#clf = clf.fit(X_train, y_train)
#clf.feature_importances_  

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2


#X, y = load_digits(return_X_y=True)
#X_new = SelectKBest(chi2, k=20).fit_transform(data, outcome['new_child'])
#X_new.shape
dict_kids = {'None': 0, 'One child': 1, 'Two children': 2, 'Three children': 3, 'Four children': 4, 'Five children': 5, 'Six children': 6}
data["aantalki2018"] = data["aantalki2018"].map(dict_kids)

feature_test = Pipeline([
               ("preprocess", preprocessor),
               ("classifier", ExtraTreesClassifier(n_estimators=50)) # "classifier", SelectKBest(chi2, k=50)
               ]) 
feature_test.fit(data,outcome['new_child'])

print_sorted_importance(feature_test['classifier'].feature_importances_, data.columns)

#2nd way
#data.columns[feature_test['classifier'].get_support(indices=True)].tolist()
#feature_test["classifier"].get_feature_names_out()
#feature_test["classifier"].columns[selector.get_support(indices=True)]

In [ ]:
# Select predictors: education, year of birth, gender, number of children in the household 
# You can do this automatically (not necessarily better): https://scikit-learn.org/stable/modules/feature_selection.html
keepcols = ['positie2019','positie2018','gebjaar', 'geslacht','aantalhh2019', 'sted2019', 
            'nettohh_f2019', 'oplmet2019', 'herkomstgroep2019', 'cf19l128','cf19l129',
            'cf19l130', 'cf19l131','cf19l132','woning2019', 'woning2018', 
            'cf19l456', 'cf19l457', 'cf19l458', 'cf19l459', 'cw19l522', 'cr19l143', 
            'cf19l483', 'cf19l484', 'cf19l485', 'cf19l486', 'cf19l487', 'cf19l488',
           'wave2008', 'wave2014', 'wave2019','aantalki2017','aantalki2018',
            'partner2018','partner2019', 'belbezig2019','belbezig2018','ch19l178',
           'cp19k118', 'cp19k021', 'cp19k056']
# Took out most mental health vars b/c seemed to make model worse 'ch19l004','ch19l014','ch19l219'
#taking out 'cr19l090','cf19l133','cf19l134', 'burgstat2019','woonvorm2019', aantalki2019 changed to 2018
#Only keep wave 2008, 2014, 2019, top 2 predictors plus 2014 because peak
# gebjaar = years of birth
# geslacht = gender
# leeftijd = age in years (redundant with gebjaar)
# positie = household position, head, married to head, etc.
# aantalhh = # household members
# partner = does household head live with partner yes/no
# burgstat = civil status (married, separated, divorced, widowed)
# woonvorm = domestic situation (married, cohabitation w/ or w/out kids, single)
# aantalki = # children of household head
# woning = type of house (own, rent, etc)
# nettohh = household income in euros
# sted = urbannness - urban - rural
# belbezig = occupation (paid, searching, etc)
# oplmet = highest education acheived
# herkomstgroep = Dutch, immigrant
# cf19l128 = children intentions
# cf19l129 = # children intentions
# cf19l130 = how soon children intentions
# cf19l131 = household help father
# cf19l132 = household help mother
# cf19l133 = childcare help father -- missing for ~2/3
# cf19l134 = childcare help mother -- missing for ~2/3
# cf19l456-cf191459 = birth year of 1st, 2nd, 3rd, 4th child, many missing (deal w/NAs below)
# cf19l483 = equality of prepping food
# cf19l484 = equality of laundry
# cf19l485 = equality of house cleaning
# cf19l486 = equality of odd jobs
# cf19l487 = equality of finances
# cf19l488 = equality of groceries
# cw19l522 = unemployment
# cr19l143 = belonging to church/religious community
# cr19l090 = what non-dutch language is spoken at home #very few observations
# wave gives year/month that was answered--want to change to categorical participated in the wave/did not
# ch19l004: general health
# ch19l014: feelings of depression
# ch19l178: being on anti-depressant meds
# ch19l219: seeing gynaecologist
# cp19k118: importance of family security 
# cp19k021: feel little concern for others
# cp19k056: take time out for others

data2 = data.loc[:, keepcols]

X_train, X_test, y_train, y_test = train_test_split(data2,
                                                    outcome,
                                                    test_size=0.25, random_state=2023)
y_train = y_train["new_child"]
y_test = y_test["new_child"]

X_train.head()

## Upsample data

In [ ]:
#Upsample data to over-represent births in training data set
from sklearn.utils import resample
target_upsample, data_upsample = resample(y_train[y_train == 1],
                                          X_train[y_train == 1],
             replace=True,
             n_samples=int(len(y_train[y_train == 0])),
             random_state=42)
#add upsample to dataset
y_train2 = pd.concat([target_upsample, y_train[y_train==0]])
X_train2 = pd.concat([data_upsample, X_train[y_train==0]])


# 3. Pre-process and model
You may not want to include the preprocessing in the pipeline if it becomes too cumbersome

Make sure to use the scoring that you want to optimize in the search space

In [864]:
# An example of a preprocessing apart from the pipeline

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler

#Convert # children string to number
dict_kids = {'None': 0, 'One child': 1, 'Two children': 2, 'Three children': 3, 'Four children': 4, 'Five children': 5, 'Six children': 6}
X_train2["aantalki2018"] = X_train2["aantalki2018"].map(dict_kids)
X_train2["aantalki2017"] = X_train2["aantalki2017"].map(dict_kids)
#2019 --> 2018

#Convert wave number to string, code NAs
#wave_cols = ['wave2007', 'wave2008', 'wave2009', 'wave2010', 'wave2011', 'wave2012', 
#             'wave2013', 'wave2014', 'wave2015', 'wave2016', 'wave2017', 'wave2018', 'wave2019']
wave_cols = ['wave2008', 'wave2014', 'wave2019']#
X_train2[wave_cols] = X_train2[wave_cols].fillna(0)
X_train2[wave_cols] = X_train2[wave_cols].astype('object')

#Create new variables about changes in partner status or number of kids (indicators)
X_train2['change_kids'] = (X_train2['aantalki2018'].fillna(-1) != X_train2['aantalki2017'].fillna(-1)) & (~X_train2['aantalki2018'].isna()) & (~X_train2['aantalki2017'].isna())
#Change in partner status
X_train2['change_partner'] = (X_train2['partner2019'].fillna(-1) != X_train2['partner2018'].fillna(-1)) & (~X_train2['partner2019'].isna()) & (~X_train2['partner2018'].isna())
#Change in household position
X_train2['change_householdPos'] = (X_train2['positie2019'].fillna(-1) != X_train2['positie2018'].fillna(-1)) & (~X_train2['positie2019'].isna()) & (~X_train2['positie2018'].isna())
#Change in employment
X_train2['change_jobs'] = (X_train2['belbezig2019'].fillna(-1) != X_train2['belbezig2018'].fillna(-1)) & (~X_train2['belbezig2019'].isna()) & (~X_train2['belbezig2018'].isna())
#Change in housing
X_train2['change_house'] = (X_train2['woning2019'].fillna(-1) != X_train2['woning2018'].fillna(-1)) & (~X_train2['woning2019'].isna()) & (~X_train2['woning2018'].isna())

# make sure missing child birth age is filled with different value
child_cols = ['cf19l456', 'cf19l457','cf19l458','cf19l459']
X_train2[child_cols] = X_train2[child_cols].fillna(0)
X_train2[child_cols] = X_train2[child_cols].astype('object')


#Suggests that this might lead to a lot of debugging (not sure of alternatives though)
# Create transformers

#import sklearn_pandas
#from sklearn_pandas import CategoricalImputer
#>>> data = np.array(['a', 'b', 'b', np.nan], dtype=object)
#>>> imputer = CategoricalImputer()
#>>> imputer.fit_transform(data)
#array(['a', 'b', 'b', 'b'], dtype=object)
# Imputer are sometimes not necessary
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), #applies most frequent category, not ideal #SimpleImputer(strategy='most_frequent')
    ('encoder', OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=50))]) 
#put variables with too few observations in separate category

numerical_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=100)), #mean is not ideal, impute based on other vars #SimpleImputer(strategy="mean")
    ('scaler', StandardScaler())])#MinMaxScaler()

#data["Age"] = data["Age"].interpolate(method='linear', limit_direction='forward', axis=0)


# Use ColumnTransformer to apply the transformations to the correct columns in the dataframe
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, selector(dtype_exclude=object)(X_train2)),
        ('cat', categorical_transformer, selector(dtype_include=object)(X_train2))])

#X_train2.head(n=10)
#X_train2['change_jobs'].value_counts()
X_train2.dtypes

positie2019             object
positie2018             object
gebjaar                  int64
geslacht                object
aantalhh2019            object
sted2019                object
nettohh_f2019          float64
oplmet2019              object
herkomstgroep2019       object
cf19l128               float64
cf19l129               float64
cf19l130               float64
cf19l131               float64
cf19l132               float64
woning2019              object
woning2018              object
cf19l456                object
cf19l457                object
cf19l458                object
cf19l459                object
cw19l522               float64
cr19l143               float64
cf19l483               float64
cf19l484               float64
cf19l485               float64
cf19l486               float64
cf19l487               float64
cf19l488               float64
wave2008                object
wave2014                object
wave2019                object
aantalki2017           float64
aantalki

In [ ]:
#Import
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_classification

# Create pipeline
model = Pipeline([
               ("preprocess", preprocessor),
               (  #GradientBoostingClassifier(n_estimators=1000, subsample=0.75, max_depth=7)) 
                   # LogisticRegression(max_iter=500)
                   "classifier", GradientBoostingClassifier(n_estimators=1000, subsample=0.75, max_depth=7, learning_rate=0.5))
                   #"classifier", HistGradientBoostingClassifier(max_iter=1000, max_depth=3, learning_rate=0.8))
               ]) 
                      
# Define the hyperparameters, this can include several classifiers, but will make it slow
# You can see different classifiers here: 
# https://scikit-learn.org/stable/supervised_learning.html#supervised-learning
#Logistic regression has values that will allow for more or less regularization (min parameters)
parameters = [
    { #'classifier': [HistGradientBoostingClassifier(max_iter=1000), GradientBoostingClassifier(n_estimators=1000, subsample=0.75)],
        'classifier': [GradientBoostingClassifier(n_estimators=1000, subsample=0.75, max_depth=7, learning_rate=0.5)], 
        #'classifier': [RandomForestClassifier()],
     #'classifier__learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
     #'classifier__max_depth': [3, 5, 7, 9]
    },
    
]


# to search over parameters 'classifier__C': np.logspace(-3, 3, 50) #regularization coefficient

# Perform hyperparameter tuning using cross-validation: https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers
# Scoring metrics: https://scikit-learn.org/stable/modules/model_evaluation.html
# f1 = f1 of the class labeled as 1 (i.e. kids)
grid_search = GridSearchCV(model, parameters, cv=5, n_jobs=-1, scoring="f1", verbose=9) #n_jobs=-1 allows for multiprocessing
grid_search.fit(X_train2, y_train2)

# Keep best model (or define it from scratch with the best coefficients found)
best_model = grid_search.best_estimator_
#model.fit(X_train2, y_train2)
#best_model = model
best_model

#First iteration: learning_rate=0.01, max_depth=7, n_estimators=1000, subsample=0.75
#Second iteration (only learning rate, max_depth): learning_rate = 0.2, max_depth = 7
#third iteration (only learning rate): learning_rate = 0.5, I think too high and will stick with default of 0.1
#new test comparing gradient and histgradient on learning rate and max depth always seems to give gradient
#some instability in parameters, but max_depth 7-9, and learning rate 0.1 -0.9?
#just optimizing on learning rate gives learning rate = 0.4
#Gives best C (complexity/regularization parameter)

In [ ]:
#Variable names in the data
#best_model
best_model["preprocess"].get_feature_names_out()

In [ ]:
def print_sorted_importance(importances, columns, codebook=None):
    for imp, var in sorted(zip(importances, columns))[::-1]:
        if codebook is not None:
            print(f"{imp:2.3f}: {codebook[var]:50.50s}")
        else:
            print(f"{imp:2.3f}: {var:50.50s}")

In [ ]:
#Feature importance (not sure exactly what it does under the hood)
# GBT from scikit-learn
print_sorted_importance(best_model['classifier'].feature_importances_, X_train2.columns)#, codebook

In [ ]:
X_test["aantalki2018"] = X_test["aantalki2018"].map(dict_kids)
X_test["aantalki2017"] = X_test["aantalki2017"].map(dict_kids)
#Create new variables about changes in partner status or number of kids (indicators)
X_test['change_kids'] = (X_test['aantalki2018'].fillna(-1) != X_test['aantalki2017'].fillna(-1)) & (~X_test['aantalki2018'].isna()) & (~X_test['aantalki2017'].isna())
#Change partner status
X_test['change_partner'] = (X_test['partner2019'].fillna(-1) != X_test['partner2018'].fillna(-1)) & (~X_test['partner2019'].isna()) & (~X_test['partner2018'].isna())
#Change in household position
X_test['change_householdPos'] = (X_test['positie2019'].fillna(-1) != X_test['positie2018'].fillna(-1)) & (~X_test['positie2019'].isna()) & (~X_test['positie2018'].isna())
#Change in employment
X_test['change_jobs'] = (X_test['belbezig2019'].fillna(-1) != X_test['belbezig2018'].fillna(-1)) & (~X_test['belbezig2019'].isna()) & (~X_test['belbezig2018'].isna())
#Change in housing
X_test['change_house'] = (X_test['woning2019'].fillna(-1) != X_test['woning2018'].fillna(-1)) & (~X_test['woning2019'].isna()) & (~X_test['woning2018'].isna())

wave_cols = ['wave2008', 'wave2014', 'wave2019']#
X_test[wave_cols] = X_test[wave_cols].fillna(0)
X_test[wave_cols] = X_test[wave_cols].astype('object')

# make sure missing child birth age is filled with different value
child_cols = ['cf19l456', 'cf19l457','cf19l458','cf19l459']
X_test[child_cols] = X_test[child_cols].fillna(0)
X_test[child_cols] = X_test[child_cols].astype('object')

In [ ]:
# permutation importance (how worse does model get without variable?)

#Feature importance via permutation
X_test["aantalki2018"] = X_test["aantalki2018"].map(dict_kids)

from sklearn.inspection import permutation_importance
r = permutation_importance(best_model, X_test, y_test,
                            n_repeats=30,
                            random_state=0)

print_sorted_importance(r["importances_mean"], X_train2.columns)

# Evaluate the model

Note: The results below are not for LogisticRegression, are for a different model

In [ ]:
X_test["aantalki2018"] = X_test["aantalki2018"].map(dict_kids)
X_test["aantalki2017"] = X_test["aantalki2017"].map(dict_kids)
#Create new variables about changes in partner status or number of kids (indicators)
X_test['change_kids'] = (X_test['aantalki2018'].fillna(-1) != X_test['aantalki2017'].fillna(-1)) & (~X_test['aantalki2018'].isna()) & (~X_test['aantalki2017'].isna())
#Change partner status
X_test['change_partner'] = (X_test['partner2019'].fillna(-1) != X_test['partner2018'].fillna(-1)) & (~X_test['partner2019'].isna()) & (~X_test['partner2018'].isna())
#Change in household position
X_test['change_householdPos'] = (X_test['positie2019'].fillna(-1) != X_test['positie2018'].fillna(-1)) & (~X_test['positie2019'].isna()) & (~X_test['positie2018'].isna())
#Change in employment
X_test['change_jobs'] = (X_test['belbezig2019'].fillna(-1) != X_test['belbezig2018'].fillna(-1)) & (~X_test['belbezig2019'].isna()) & (~X_test['belbezig2018'].isna())
#Change in housing
X_test['change_house'] = (X_test['woning2019'].fillna(-1) != X_test['woning2018'].fillna(-1)) & (~X_test['woning2019'].isna()) & (~X_test['woning2018'].isna())

wave_cols = ['wave2008', 'wave2014', 'wave2019']#
X_test[wave_cols] = X_test[wave_cols].fillna(0)
X_test[wave_cols] = X_test[wave_cols].astype('object')

# make sure missing child birth age is filled with different value
child_cols = ['cf19l456', 'cf19l457','cf19l458','cf19l459']
X_test[child_cols] = X_test[child_cols].fillna(0)
X_test[child_cols] = X_test[child_cols].astype('object')

In [ ]:
# Print ROC curve, it tells you how well you can balance false and true positives
RocCurveDisplay.from_predictions(
    y_test,
    best_model.predict_proba(X_test)[:, 1],
    color="cornflowerblue",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
# Create predictions
y_pred = best_model.predict(X_test)

# Report classification table
print(classification_report(y_test, y_pred))

# rerun model on all given data

In [ ]:
#model.fit(X_train2, y_train2)
#best_model = model
#not for now

# Save model

In [ ]:
import os
os.makedirs("../models", exist_ok=True)

# Dump model (don't change the name)
dump(best_model, "../models/model.joblib") #allows you to save and load model

# For submission

In [ ]:
test_data = pd.read_csv("datasets/LISS_test.csv", encoding="cp1252")

In [ ]:
def predict_outcomes(df):
    """Process the input data and write the predictions."""
    # Dictionary used
    dict_kids = {'None': 0, 'One child': 1, 'Two children': 2, 'Three children': 3, 'Four children': 4, 'Five children': 5, 'Six children': 6}
    
    # Keep 
    keepcols = ['positie2019','positie2018','gebjaar', 'geslacht','aantalhh2019', 'sted2019', 
            'nettohh_f2019', 'oplmet2019', 'herkomstgroep2019', 'cf19l128','cf19l129',
            'cf19l130', 'cf19l131','cf19l132','woning2019', 'woning2018', 
            'cf19l456', 'cf19l457', 'cf19l458', 'cf19l459', 'cw19l522', 'cr19l143', 
            'cf19l483', 'cf19l484', 'cf19l485', 'cf19l486', 'cf19l487', 'cf19l488',
           'wave2008', 'wave2014', 'wave2019','aantalki2017','aantalki2018',
            'partner2018','partner2019', 'belbezig2019','belbezig2018','ch19l178',
           'cp19k118', 'cp19k021', 'cp19k056']
    results = df[["nomem_encr"]]
    
    df = df.loc[:, keepcols]
    
    df["aantalki2018"] = df["aantalki2018"].map(dict_kids)
    df["aantalki2017"] = df["aantalki2017"].map(dict_kids)
    #Create new variables about changes in partner status or number of kids (indicators)
    df['change_kids'] = (df['aantalki2018'].fillna(-1) != df['aantalki2017'].fillna(-1)) & (~df['aantalki2018'].isna()) & (~df['aantalki2017'].isna())
    #Change partner status
    df['change_partner'] = (df['partner2019'].fillna(-1) != df['partner2018'].fillna(-1)) & (~df['partner2019'].isna()) & (~df['partner2018'].isna())
    #Change in household position
    df['change_householdPos'] = (df['positie2019'].fillna(-1) != df['positie2018'].fillna(-1)) & (~df['positie2019'].isna()) & (~df['positie2018'].isna())
    #Change in employment
    df['change_jobs'] = (df['belbezig2019'].fillna(-1) != df['belbezig2018'].fillna(-1)) & (~df['belbezig2019'].isna()) & (~df['belbezig2018'].isna())
    #Change in housing
    df['change_house'] = (df['woning2019'].fillna(-1) != df['woning2018'].fillna(-1)) & (~df['woning2019'].isna()) & (~df['woning2018'].isna())

    wave_cols = ['wave2008', 'wave2014', 'wave2019']#
    df[wave_cols] = df[wave_cols].fillna(0)
    df[wave_cols] = df[wave_cols].astype('object')

    # make sure missing child birth age is filled with different value
    child_cols = ['cf19l456', 'cf19l457','cf19l458','cf19l459']
    df[child_cols] = df[child_cols].fillna(0)
    df[child_cols] = df[child_cols].astype('object')
                            
    # Load your trained model from the models directory
    model_path = os.path.join(os.path.dirname(__file__), "..", "models", "model.joblib")
    model = load(model_path)

    # Use your trained model for prediction
    results.loc[:, "prediction"] = model.predict(df)

    #If you use predict_proba to get a probability and a different threshold
    #df["prediction"] = (df["prediction"] >= 0.5).astype(int)
    return results

In [ ]:
__file__ = './' #this is not needed outside juypter notebooks
predict_outcomes(test_data)#original_data